In [1]:
%pip install newsapi-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from newsapi import NewsApiClient
import pandas as pd
from sqlalchemy import create_engine, types, text
import datetime
from datetime import datetime
#import pyodbc

In [2]:
# Establish a connection to the SQL Server database
server = 'server'  # Replace 'your_server_name' with your SQL Server instance name
database = 'database'  # Replace 'your_database_name' with your database name
username = 'username'  # Replace 'your_username' with your SQL Server username
password = 'password'  # Replace 'your_password' with your SQL Server password
driver = 'driver'  # Replace with your SQL Server driver (e.g., 'ODBC+Driver+17+for+SQL+Server')
port = 1433 # postgresql common port change it for yourselves

In [3]:
conn_str = f'mssql+pyodbc://{username}:{password}@{server}:{port}/{database}?driver={driver}'
engine = create_engine(conn_str, isolation_level='AUTOCOMMIT')

In [4]:
#conn = pyodbc.connect(conn_str)
##cursor = conn.cursor()

In [4]:
# Init
newsapi = NewsApiClient(api_key='bd2b8887d4a84984b5436bd30d7a0a37')

In [6]:
#sources = newsapi.get_sources()

#df_sources1 = pd.DataFrame(sources['sources'])

#df_sources2 = df_sources1[df_sources1['language'] == 'en']

#df_ids = df_sources2[['id']].drop_duplicates()

#df_ids.to_csv('sources.csv', index=False)

In [5]:
sources = newsapi.get_sources()

df_sources = pd.DataFrame(sources['sources'])

#a = df_sources['category'].drop_duplicates()

#technology = df_sources[df_sources['category'] == 'business']

#technology

In [9]:
q_list = ['economy', 'business', 'money', 'tech', 'science']

dataframes = {}

listt = []

#interest = ['']

for i in range(len(q_list)): 
    all_articles_from_sources = newsapi.get_everything( 
        q=q_list[i], 
        sources='bbc-news,bloomberg,the-verge,abc-news,google-news,business-insider,cnn,the-wall-street-journal,the-washington-times', 
        language='en', 
        from_param='2025-01-02', 
        to='2025-02-03', 
        sort_by='relevancy' )

    articles = all_articles_from_sources['articles']
    dataframes[q_list[i]] = pd.DataFrame(articles)
    listt.append(dataframes[q_list[i]])

df_total = pd.concat(listt, axis=0)
#df_total.to_csv('last_one3.csv', index=False)

In [ ]:
#df_total.sort_values('publishedAt', ascending=False, inplace=True)

In [ ]:
#df_total

In [8]:
interests = ['Russia', 'Ukraine', 'Chinese', 'America', 'Erdogan', 'Erdoğan', 'Turkey', 'inflation', 'economy', 'NASA', 'Syria', 'Donald Trump', 'AWS', 'Google']

df_total['interest'] = df_total['description'].apply(lambda x: next((i for i in interests if i in x), 'Others'))

In [9]:
df_total['publishedAt'] = pd.to_datetime(df_total['publishedAt']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [28]:
#df_total['source'] = df_total['source'].astype(str)

In [10]:
df_total['source_name'] = [i.get('name') if isinstance(i, dict) else None for i in df_total['source']]

In [ ]:
#df_total['publishedAt'] = pd.to_datetime(df_total['publishedAt'])

## IN SQL IF YOU WANNA FORMAT THIS YOU CAN USE THIS BELOW. THAT UGLY FORMATTING IS SMHTN.

# SELECT source, author, title, description, url, urlToImage, FORMAT(publishedAt, 'yyyy-MM-dd') AS publishedAt, content, interest, source_name FROM [dbo].[News]
# ORDER BY publishedAt;

In [20]:
df_total['source'] = df_total['source'].astype(str)

In [ ]:
df_total

In [14]:
# df_total['source_id'] = df_total['source'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
# df_total['source_name'] = df_total['source'].apply(lambda x: x.get('name') if isinstance(x, dict) else None)
# df_total = df_total.drop(columns=['source'])

In [ ]:
# asdd = df_total[df_total['interest'] != 'Null']

# asdd

In [24]:
# IF YOU WANNA CREATE A NEW DATABASE THAT NOT EXIST

newdatabase = 'News_Database'


check_query = f"SELECT database_id FROM sys.databases WHERE name = '{newdatabase}'"

create_query = f"CREATE DATABASE {newdatabase}"

with engine.connect() as connection:
    result = connection.execute(text(check_query))
    if result.fetchone() is None:
        connection.execute(text(create_query))
    else:
        print("Database already exists.")

Database already exists.


In [25]:
print(pd.io.sql.get_schema(df_total, name="News", con=engine))


CREATE TABLE [News] (
	source VARCHAR(max) NULL, 
	author VARCHAR(max) NULL, 
	title VARCHAR(max) NULL, 
	description VARCHAR(max) NULL, 
	url VARCHAR(max) NULL, 
	[urlToImage] VARCHAR(max) NULL, 
	[publishedAt] DATETIME NULL, 
	content VARCHAR(max) NULL, 
	interest VARCHAR(max) NULL, 
	source_name VARCHAR(max) NULL
)




In [26]:
df_total.to_sql(name='News', con=engine, if_exists='replace')

120